In [14]:
import os
from openai import OpenAI
import openai
from pydantic_settings import BaseSettings, SettingsConfigDict
from pathlib import Path

In [4]:
ENV_PATH = Path("../.envs/dev.env")

In [7]:
class Settings(BaseSettings):
    OPENAI_API_KEY: str
    HF_TOKEN: str
    LLAMAPARSE_API_TOKEN: str

    model_config = SettingsConfigDict(
        env_file=ENV_PATH if ENV_PATH.exists() else None,
        env_file_encoding="utf-8",
        extra="ignore",
        case_sensitive=False
    )


In [8]:
settings = Settings()

In [11]:
client = OpenAI(api_key=settings.OPENAI_API_KEY,)

In [12]:
# --- Block 2: First Turn - Asking the Initial Question ---
print("\n--- Starting Conversation: Turn 1 ---")

# Define the system message (persona) for the AI Tutor
system_message = {"role": "system", "content": "You are a helpful AI Tutor explaining Large Language Model concepts simply."}

# Define the user's first question
user_message_1 = {"role": "user", "content": "Can you explain what 'tokens' are in the context of LLMs, like I'm new to this?"}

# Create the messages list for the *first* API call
messages_history = [
    system_message,
    user_message_1
]

print(f"Sending messages: {messages_history}")


--- Starting Conversation: Turn 1 ---
Sending messages: [{'role': 'system', 'content': 'You are a helpful AI Tutor explaining Large Language Model concepts simply.'}, {'role': 'user', 'content': "Can you explain what 'tokens' are in the context of LLMs, like I'm new to this?"}]


In [13]:
# Define parameters for this call
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.5 # Slightly creative but still grounded explanation
MAX_TOKENS = 150  # Limit the length of the explanation
SEED = 123        # Make this explanation reproducible

In [15]:
try:
    print(f"\nMaking API call to {MODEL}...")
    # Use the client object's method to create a chat completion
    completion_1 = client.chat.completions.create(
        model=MODEL,
        messages=messages_history,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        seed=SEED
    )
    print("API call successful.")

    # --- Process the response from the first turn ---
    # Extract the assistant's reply content
    assistant_response_1 = completion_1.choices[0].message.content
    # Extract the full message object to add to history later
    assistant_message_1 = completion_1.choices[0].message

    print("\nAI Tutor (Turn 1):")
    print(assistant_response_1)

    # Print token usage for this call
    usage_1 = completion_1.usage
    print(f"\nToken Usage (Turn 1): Prompt={usage_1.prompt_tokens}, Completion={usage_1.completion_tokens}, Total={usage_1.total_tokens}")
    finish_reason_1 = completion_1.choices[0].finish_reason
    print(f"Finish Reason: {finish_reason_1}")

except openai.APIError as e:
    # Handle API errors (e.g., server issues, rate limits)
    print(f"OpenAI API returned an API Error: {e}")
except openai.AuthenticationError as e:
    # Handle Authentication errors (e.g., invalid API key)
    print(f"OpenAI Authentication Error: {e}")
except Exception as e:
    # Handle other potential errors
    print(f"An unexpected error occurred: {e}")


Making API call to gpt-4o-mini...
API call successful.

AI Tutor (Turn 1):
Sure! In the context of Large Language Models (LLMs), a "token" is a basic unit of text that the model processes. Tokens can be words, parts of words, or even punctuation marks. 

Think of it this way: when you read a sentence, you break it down into smaller pieces to understand it better. Similarly, LLMs break down text into tokens to analyze and generate language.

For example, the sentence "I love apples!" might be broken down into the following tokens:
- "I"
- "love"
- "apples"
- "!"

In some cases, a token might represent just a part of a word, especially for longer or complex words. For instance, the word "unhappiness"

Token Usage (Turn 1): Prompt=46, Completion=150, Total=196
Finish Reason: length


In [16]:
# Parameters for the second call (could be the same or different)
# Let's make it slightly more deterministic for a factual answer
TEMPERATURE_2 = 0.2
MAX_TOKENS_2 = 100
# Using the same seed ensures the *entire conversation flow* is reproducible if inputs are identical
SEED_2 = 123

In [ ]:
try:
    print(f"\nMaking API call to {MODEL} (Turn 2)...")
    completion_2 = client.chat.completions.create(
        model=MODEL,
        messages=messages_history, # Send the *full* history
        temperature=TEMPERATURE_2,
        max_tokens=MAX_TOKENS_2,
        seed=SEED_2
    )
    print("API call successful.")

    # --- Process the response from the second turn ---
    assistant_response_2 = completion_2.choices[0].message.content
    # We don't strictly need to save assistant_message_2 unless continuing the conversation

    print("\nAI Tutor (Turn 2):")
    print(assistant_response_2)

    # Print token usage for this call
    usage_2 = completion_2.usage
    print(f"\nToken Usage (Turn 2): Prompt={usage_2.prompt_tokens}, Completion={usage_2.completion_tokens}, Total={usage_2.total_tokens}")
    # Note: prompt_tokens for turn 2 will be larger as it includes the history from turn 1.
    finish_reason_2 = completion_2.choices[0].finish_reason
    print(f"Finish Reason: {finish_reason_2}")

except openai.APIError as e:
    print(f"OpenAI API returned an API Error: {e}")
except openai.AuthenticationError as e:
    print(f"OpenAI Authentication Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")